# Разработка модели детекции наклеек

In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

## Загрузка предобученной модели Faster R-CNN

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

## Пользовательский классификатор

In [ ]:
num_classes = 2  # фон и наклейка
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

## Загрузка и предобработка данных

In [ ]:
import os
import torch
from PIL import Image
import json

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, annotations, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.annotations = list(sorted(os.listdir(os.path.join(root, "annotations"))))

    def __getitem__(self, idx):
        # Загрузка изображений и аннотаций
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        ann_path = os.path.join(self.root, "annotations", self.annotations[idx])

        img = Image.open(img_path).convert("RGB")

        # Загрузка файла аннотации
        with open(ann_path) as f:
            boxes = json.load(f)["boxes"]

        # Конвертация координат боксов в тензор Torch
        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        num_objs = len(boxes)
        # Пометка класса наклейки как 1 (предполагая, что 0 - это фон)
        labels = torch.ones((num_objs,), dtype=torch.int64)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
import torchvision.transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        # Дополнительные трансформации для обучения
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

## Обучение модели

In [ ]:
from engine import train_one_epoch, evaluate
import utils

def train_model(model, data_loader, optimizer, device, num_epochs=3):
    for epoch in range(num_epochs):
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        lr_scheduler.step()

        # оценка на каждом шаге
        evaluate(model, data_loader, device=device)


In [ ]:
# Инициализация датасета, DataLoader и модели
dataset = CustomDataset(root="path/to/images", annotations="path/to/annotations", transforms=get_transform(train=True))
data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=utils.collate_fn)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Инициализация оптимизатора
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Запуск обучения
train_model(model, data_loader, optimizer, device)


## Сохранение модели

In [ ]:
torch.save(model.state_dict(), 'fasterrcnn_resnet50_fpn.pth')